In [ ]:
!pip install onnx

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx

# Простейшая кастомная нейросеть для классификации изображений (например, 1x28x28)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Первый сверточный слой: вход 1 канал, выход 8 каналов, ядро 3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)

        # Второй сверточный слой: вход 8 каналов, выход 16 каналов, ядро 3x3
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(16 * 7 * 7, 10)  # выход 10 классов

    def forward(self, x):
        # Первый сверточный слой + ReLU + MaxPool
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)  # уменьшаем размер в 2 раза

        # Второй сверточный слой + ReLU + MaxPool
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)  # снова уменьшаем

        # Преобразуем тензор в вектор перед полносвязным слоем
        x = x.view(x.size(0), -1)

        # Полносвязный слой
        x = self.fc1(x)
        return x

# Создание экземпляра модели
model = SimpleCNN()
model.eval()  # Переводим модель в режим оценки

# Пример входных данных (один чёрно-белый 28x28 "образ")
dummy_input = torch.randn(1, 1, 28, 28)

# Экспорт модели в формат ONNX
torch.onnx.export(
    model,                        # модель
    dummy_input,                  # пример входа
    "simple_cnn.onnx",           # имя выходного файла
    input_names=['input'],        # имя входа
    output_names=['output'],      # имя выхода
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},  # поддержка разных batch size
    opset_version=11              # версия ONNX (11 — совместимая)
)

print("Модель успешно экспортирована в ONNX.")

In [ ]:
# Новая модель

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx

class MyAwesomeCNN(nn.Module):
    def __init__(self):
        super(MyAwesomeCNN, self).__init__()

        # Вход: [batch, 1, 28, 28]
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        # После conv1: [batch, 16, 28, 28]
        # После MaxPool: [batch, 16, 14, 14]

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # После conv2: [batch, 32, 14, 14]
        # После MaxPool: [batch, 32, 7, 7]

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # После conv3: [batch, 64, 7, 7]
        # После MaxPool: [batch, 64, 3, 3]

        # "Выпрямляем" тензор [batch, 64, 3, 3] в вектор
        self.fc1 = nn.Linear(in_features=64 * 3 * 3, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10) # 10 классов на выходе

    def forward(self, x):
        # Блок 1
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Блок 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Блок 3
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # Переход к классификатору
        x = x.view(x.size(0), -1) # Flatten

        # Полносвязные слои
        x = self.fc1(x)
        x = F.relu(x) # Активация между полносвязными слоями
        x = self.fc2(x)

        return x

# Экспорт
my_model = MyAwesomeCNN()
my_model.eval()

dummy_input = torch.randn(1, 1, 28, 28)

onnx_file_name = "my_new_cnn.onnx"
torch.onnx.export(
    my_model,
    dummy_input,
    onnx_file_name,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    opset_version=11
)

print(f"Модель '{onnx_file_name}' успешно экспортирована.")